In [1]:
from UCLSE.environment import Market_session, yamlLoad
from UCLSE.test.utils import identical_replay_vars,side_by_side_period_by_period_difference_checker
from UCLSE.exchange import Exchange
import copy

I have extensively rewritten the exchange.py file to account for variable quantity orders. The main complication of this that any new order can cause a greater number of outcomes. Consider a new bid. It can have the following effects:

- 0 Add itself to the LOB and do nothing
- 1 Bid exactly fills a best bid ask, two orders are deleted off LOB and two traders informed.
- 2a Bid completely fills with any number of ask orders at any number of prices. Any number of orders are deleted and any number of traders are informed.
- 2a As above, but only partially fill an existing ask on the LOB. Ask order owner needs to be made aware of order change.
- 3 Bid Partially filled - As above but bid owner needs to be made aware of change. 
<br>
There are also bookkeeping considerations which I haven't considered yet. 

But, to check the ammendments haven't messed up the original calculations, we should run parallel experiments using the old and new exchange methods to ensure that they result in the same thing. 

In [2]:
import os
pa=os.getcwd()
config_name='config\\mkt_cfg.yml'
config_path=os.path.join(pa,config_name)
environ_dic=yamlLoad(config_path)

#just updates the tradefile every time there is a trade
environ_dic['dump_each_trade']=True
environ_dic['verbose']=False
environ_dic['end_time']=50

I have add process_order as an attribute of the session object, to allow selection of which process order mechanism we use. It will default to the old one.

First check that the new method works. Begin by instantiating a Market_session and then copying it exactly.

In [3]:
import copy
sess=Market_session(**environ_dic)
sess.process_order=sess.exchange.process_order3w
sess1=copy.deepcopy(sess)


Then we can run the first simulation, store the events and push those events into the second simulation session to make sure that the end result is the same.

In [4]:
sess.simulate(sess.trade_stats_df3,recording=True)
sess1.simulate(sess1.trade_stats_df3,replay_vars=sess.replay_vars,recording=True)
(sess1.df.fillna(0)==sess.df.fillna(0)).all() #note that the best_bid or best_bid do not always give a True value. Because nan!=nan.

Bid  leg 0  lifts best  Ask 105
counterparty S23 price 95
[{'time': 1.8749999999999953, 'p2_qid': 4.0, 'price': 95, 'type': 'Trade', 'party1': 'S23', 'qty': 1, 'p1_qid': 3, 'party2': 'B11'}]
Bid  leg 0  lifts best  Ask 105
counterparty S35 price 97
[{'time': 5.312500000000042, 'p2_qid': 17.0, 'price': 97, 'type': 'Trade', 'party1': 'S35', 'qty': 1, 'p1_qid': 16, 'party2': 'B02'}]
Bid  leg 0  lifts best  Ask 99
counterparty S20 price 97
[{'time': 6.2125000000000545, 'p2_qid': 22.0, 'price': 97, 'type': 'Trade', 'party1': 'S20', 'qty': 1, 'p1_qid': 19, 'party2': 'B24'}]
Bid  leg 0  lifts best  Ask 106
counterparty S18 price 95
[{'time': 8.250000000000066, 'p2_qid': 49.0, 'price': 95, 'type': 'Trade', 'party1': 'S18', 'qty': 1, 'p1_qid': 26, 'party2': 'B27'}]
Bid  leg 0  lifts best  Ask 106
counterparty S31 price 95
[{'time': 9.187500000000012, 'p2_qid': 62.0, 'price': 95, 'type': 'Trade', 'party1': 'S31', 'qty': 1, 'p1_qid': 36, 'party2': 'B12'}]
Bid  leg 0  lifts best  Ask 106
counterpa

counterparty B16 price 101
[{'time': 33.87499999999914, 'p2_qid': 587.0, 'price': 101, 'type': 'Trade', 'party1': 'B16', 'qty': 1, 'p1_qid': 580, 'party2': 'S37'}]
Ask  leg 0  lifts best  Bid 100
counterparty B33 price 100
[{'time': 34.374999999999254, 'p2_qid': 601.0, 'price': 100, 'type': 'Trade', 'party1': 'B33', 'qty': 1, 'p1_qid': 572, 'party2': 'S38'}]
Ask  leg 0  lifts best  Bid 100
counterparty B31 price 100
[{'time': 34.47499999999928, 'p2_qid': 605.0, 'price': 100, 'type': 'Trade', 'party1': 'B31', 'qty': 1, 'p1_qid': 576, 'party2': 'S32'}]
Ask  leg 0  lifts best  Bid 98
counterparty B29 price 99
[{'time': 35.4499999999995, 'p2_qid': 624.0, 'price': 99, 'type': 'Trade', 'party1': 'B29', 'qty': 1, 'p1_qid': 606, 'party2': 'S25'}]
Ask  leg 0  lifts best  Bid 101
counterparty B23 price 101
[{'time': 35.79999999999958, 'p2_qid': 631.0, 'price': 101, 'type': 'Trade', 'party1': 'B23', 'qty': 1, 'p1_qid': 626, 'party2': 'S04'}]
Bid  leg 0  lifts best  Ask 108
counterparty S08 price 

expid                    True
time                     True
GVWY      balance_sum    True
          n              True
          pc             True
SHVR      balance_sum    True
          n              True
          pc             True
ZIC       balance_sum    True
          n              True
          pc             True
ZIP       balance_sum    True
          n              True
          pc             True
best_bid                 True
best_ask                 True
dtype: bool

This function checks that the two replay_vars attributes are the same and returns the difference if it exists:
    

In [5]:
identical_replay_vars(sess,sess1)

no differences found


{}

In [26]:
sess.exchange.tape

[{'p1_qid': 2,
  'p2_qid': 9.0,
  'party1': 'S07',
  'party2': 'B18',
  'price': 95,
  'qty': 1,
  'time': 2.525000000000002,
  'type': 'Trade'},
 {'p1_qid': 33,
  'p2_qid': 35.0,
  'party1': 'B13',
  'party2': 'S11',
  'price': 105,
  'qty': 1,
  'time': 4.937500000000036,
  'type': 'Trade'},
 {'p1_qid': 57,
  'p2_qid': 63.0,
  'party1': 'B15',
  'party2': 'S10',
  'price': 105,
  'qty': 1,
  'time': 7.587500000000074,
  'type': 'Trade'},
 {'p1_qid': 60,
  'p2_qid': 68.0,
  'party1': 'B04',
  'party2': 'S00',
  'price': 105,
  'qty': 1,
  'time': 8.037500000000078,
  'type': 'Trade'},
 {'p1_qid': 72,
  'p2_qid': 83.0,
  'party1': 'B11',
  'party2': 'S37',
  'price': 105,
  'qty': 1,
  'time': 8.987500000000024,
  'type': 'Trade'},
 {'p1_qid': 137,
  'p2_qid': 151.0,
  'party1': 'B17',
  'party2': 'S35',
  'price': 106,
  'qty': 1,
  'time': 13.487499999999768,
  'type': 'Trade'},
 {'p1_qid': 145,
  'p2_qid': 162.0,
  'party1': 'B26',
  'party2': 'S06',
  'price': 106,
  'qty': 1,
  't

For a more forensic approach to difference checking, this function will look for differences at the end of every period and stop if one occurs.

In [6]:
sess=Market_session(**environ_dic)
sess.process_order=sess.exchange.process_order3w
sess1=copy.deepcopy(sess)
side_by_side_period_by_period_difference_checker(sess,sess1)


Ask  leg 0  lifts best  Bid 105
counterparty B38 price 105
[{'time': 5.18750000000004, 'p2_qid': 62.0, 'price': 105, 'type': 'Trade', 'party1': 'B38', 'qty': 1, 'p1_qid': 30, 'party2': 'S05'}]
Ask  leg 0  lifts best  Bid 105
counterparty B38 price 105
[{'time': 5.18750000000004, 'p2_qid': 62.0, 'price': 105, 'type': 'Trade', 'party1': 'B38', 'qty': 1, 'p1_qid': 30, 'party2': 'S05'}]
Ask  leg 0  lifts best  Bid 105
counterparty B27 price 105
[{'time': 5.312500000000042, 'p2_qid': 63.0, 'price': 105, 'type': 'Trade', 'party1': 'B27', 'qty': 1, 'p1_qid': 31, 'party2': 'S24'}]
Ask  leg 0  lifts best  Bid 105
counterparty B27 price 105
[{'time': 5.312500000000042, 'p2_qid': 63.0, 'price': 105, 'type': 'Trade', 'party1': 'B27', 'qty': 1, 'p1_qid': 31, 'party2': 'S24'}]
Ask  leg 0  lifts best  Bid 95
counterparty B28 price 105
[{'time': 6.125000000000053, 'p2_qid': 77.0, 'price': 105, 'type': 'Trade', 'party1': 'B28', 'qty': 1, 'p1_qid': 74, 'party2': 'S19'}]
Ask  leg 0  lifts best  Bid 95
co

Ask  leg 0  lifts best  Bid 97
counterparty B02 price 103
[{'time': 24.112499999999166, 'p2_qid': 440.0, 'price': 103, 'type': 'Trade', 'party1': 'B02', 'qty': 1, 'p1_qid': 430, 'party2': 'S07'}]
Ask  leg 0  lifts best  Bid 97
counterparty B02 price 103
[{'time': 24.112499999999166, 'p2_qid': 440.0, 'price': 103, 'type': 'Trade', 'party1': 'B02', 'qty': 1, 'p1_qid': 430, 'party2': 'S07'}]
Bid  leg 0  lifts best  Ask 107
counterparty S01 price 104
[{'time': 26.774999999999014, 'p2_qid': 499.0, 'price': 104, 'type': 'Trade', 'party1': 'S01', 'qty': 1, 'p1_qid': 466, 'party2': 'B10'}]
Bid  leg 0  lifts best  Ask 107
counterparty S01 price 104
[{'time': 26.774999999999014, 'p2_qid': 499.0, 'price': 104, 'type': 'Trade', 'party1': 'S01', 'qty': 1, 'p1_qid': 466, 'party2': 'B10'}]
Bid  leg 0  lifts best  Ask 105
counterparty S28 price 104
[{'time': 26.899999999999007, 'p2_qid': 503.0, 'price': 104, 'type': 'Trade', 'party1': 'S28', 'qty': 1, 'p1_qid': 467, 'party2': 'B19'}]
Bid  leg 0  lifts

[{'time': 47.16250000000216, 'p2_qid': 950.0, 'price': 105, 'type': 'Trade', 'party1': 'B29', 'qty': 1, 'p1_qid': 935, 'party2': 'S37'}]
Ask  leg 0  lifts best  Bid 99
counterparty B29 price 105
[{'time': 47.16250000000216, 'p2_qid': 950.0, 'price': 105, 'type': 'Trade', 'party1': 'B29', 'qty': 1, 'p1_qid': 935, 'party2': 'S37'}]
Bid  leg 0  lifts best  Ask 109
counterparty S39 price 109
[{'time': 47.37500000000221, 'p2_qid': 956.0, 'price': 109, 'type': 'Trade', 'party1': 'S39', 'qty': 1, 'p1_qid': 938, 'party2': 'B34'}]
Bid  leg 0  lifts best  Ask 109
counterparty S39 price 109
[{'time': 47.37500000000221, 'p2_qid': 956.0, 'price': 109, 'type': 'Trade', 'party1': 'S39', 'qty': 1, 'p1_qid': 938, 'party2': 'B34'}]
Ask  leg 0  lifts best  Bid 99
counterparty B12 price 105
[{'time': 47.700000000002284, 'p2_qid': 970.0, 'price': 105, 'type': 'Trade', 'party1': 'B12', 'qty': 1, 'p1_qid': 955, 'party2': 'S33'}]
Ask  leg 0  lifts best  Bid 99
counterparty B12 price 105
[{'time': 47.700000000

Now make one session object use the process_order and one the new process_order

In [7]:
sess=Market_session(**environ_dic)

sess1=copy.deepcopy(sess)
sess.process_order=sess.exchange.process_order3w
side_by_side_period_by_period_difference_checker(sess,sess1)

Ask  leg 0  lifts best  Bid 100
counterparty B27 price 100
[{'time': 4.162500000000025, 'p2_qid': 14.0, 'price': 100, 'type': 'Trade', 'party1': 'B27', 'qty': 1, 'p1_qid': 13, 'party2': 'S33'}]
[{'time': 4.162500000000025, 'price': 100, 'type': 'Trade', 'party1': 'B27', 'qty': 1, 'party2': 'S33'}]
Bid  leg 0  lifts best  Ask 105
counterparty S37 price 100
[{'time': 5.075000000000038, 'p2_qid': 24.0, 'price': 100, 'type': 'Trade', 'party1': 'S37', 'qty': 1, 'p1_qid': 20, 'party2': 'B32'}]
[{'time': 5.075000000000038, 'price': 100, 'type': 'Trade', 'party1': 'S37', 'qty': 1, 'party2': 'B32'}]
Bid  leg 0  lifts best  Ask 105
counterparty S00 price 101
[{'time': 5.112500000000039, 'p2_qid': 25.0, 'price': 101, 'type': 'Trade', 'party1': 'S00', 'qty': 1, 'p1_qid': 12, 'party2': 'B19'}]
[{'time': 5.112500000000039, 'price': 101, 'type': 'Trade', 'party1': 'S00', 'qty': 1, 'party2': 'B19'}]
Ask  leg 0  lifts best  Bid 95
counterparty B05 price 95
[{'time': 5.325000000000042, 'p2_qid': 28.0, '

Ask  leg 0  lifts best  Bid 98
counterparty B25 price 106
[{'time': 36.73749999999979, 'p2_qid': 708.0, 'price': 106, 'type': 'Trade', 'party1': 'B25', 'qty': 1, 'p1_qid': 700, 'party2': 'S32'}]
[{'time': 36.73749999999979, 'price': 106, 'type': 'Trade', 'party1': 'B25', 'qty': 1, 'party2': 'S32'}]
Bid  leg 0  lifts best  Ask 107
counterparty S25 price 107
[{'time': 38.01250000000008, 'p2_qid': 743.0, 'price': 107, 'type': 'Trade', 'party1': 'S25', 'qty': 1, 'p1_qid': 671, 'party2': 'B11'}]
[{'time': 38.01250000000008, 'price': 107, 'type': 'Trade', 'party1': 'S25', 'qty': 1, 'party2': 'B11'}]
Ask  leg 0  lifts best  Bid 107
counterparty B38 price 108
[{'time': 38.75000000000025, 'p2_qid': 762.0, 'price': 108, 'type': 'Trade', 'party1': 'B38', 'qty': 1, 'p1_qid': 751, 'party2': 'S28'}]
[{'time': 38.75000000000025, 'price': 108, 'type': 'Trade', 'party1': 'B38', 'qty': 1, 'party2': 'S28'}]
Ask  leg 0  lifts best  Bid 106
counterparty B06 price 108
[{'time': 38.76250000000025, 'p2_qid': 

In [19]:
identical_replay_vars(sess,sess1)

no differences found


{}

And for completion, swap it over so the new process_order works in replay mode:

In [8]:
sess=Market_session(**environ_dic)
sess.verbose=True
sess1=copy.deepcopy(sess)
sess1.process_order=sess1.exchange.process_order3w
side_by_side_period_by_period_difference_checker(sess,sess1)


trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial

replay False SHVR   0.0 []
Trader Quote: [S03 Ask P=095 Q=1 T= 0.22 QID:None OID:-3.14]
Trader Quote: [S03 Ask P=662 Q=1 T= 5.18 QID:30 OID:-3.14]

trial0001;  
replay True SHVR   0.0 []
Trader Quote: [S03 Ask P=095 Q=1 T= 0.22 QID:None OID:-3.14]
Trader Quote: [S03 Ask P=662 Q=1 T= 5.18 QID:30 OID:-3.14]

trial0001;  
replay False GVWY   0.0 []
Trader Quote: [B19 Bid P=105 Q=1 T= 0.64 QID:None OID:-3.14]
Trader Quote: [B19 Bid P=105 Q=1 T= 5.19 QID:31 OID:-3.14]

trial0001;  
replay True GVWY   0.0 []
Trader Quote: [B19 Bid P=105 Q=1 T= 0.64 QID:None OID:-3.14]
Trader Quote: [B19 Bid P=105 Q=1 T= 5.19 QID:31 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [S03 Ask P=095 Q=1 T= 0.22 QID:None OID:-3.14]
Trader Quote: [S03 Ask P=661 Q=1 T= 5.21 QID:32 OID:-3.14]

trial0001;  
replay True SHVR   0.0 []
Trader Quote: [S03 Ask P=095 Q=1 T= 0.22 QID:None OID:-3.14]
Trader Quote: [S03 Ask P=661 Q=1 T= 5.21 QID:32 OID:-3.14]

trial0001;  

trial000

replay True SHVR   0.0 []
Trader Quote: [B06 Bid P=105 Q=1 T= 2.70 QID:None OID:-3.14]
Trader Quote: [B06 Bid P=105 Q=1 T= 7.79 QID:65 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B01 Bid P=105 Q

replay True ZIC   0.0 []
Trader Quote: [S17 Ask P=095 Q=1 T= 5.19 QID:None OID:-3.14]
Trader Quote: [S17 Ask P=591 Q=1 T=10.10 QID:101 OID:-3.14]

trial0001;  
replay False ZIP   0.0 []
Trader Quote: [B38 Bid P=105 Q=1 T= 3.42 QID:None OID:-3.14]
Trader Quote: [B38 Bid P=093 Q=1 T=10.11 QID:102 OID:-3.14]

trial0001;  
replay True ZIP   0.0 []
Trader Quote: [B38 Bid P=105 Q=1 T= 3.42 QID:None OID:-3.14]
Trader Quote: [B38 Bid P=093 Q=1 T=10.11 QID:102 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B21 Bid P=106 Q=1 T= 8.23 QID:None OID:-3.14]
Trader Quote: [B21 Bid P=000 Q=1 T=10.25 QID:103 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B21 Bid P=106 Q=1 T= 8.23 QID:None OID

replay True GVWY   0.0 []
Trader Quote: [B10 Bid P=106 Q=1 T=11.28 QID:None OID:-3.14]
Trader Quote: [B10 Bid P=106 Q=1 T=11.54 QID:123 OID:-3.14]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S32 Ask P=095 Q=1 T= 2.66 QID:None OID:-3.14]
Trader Quote: [S32 Ask P=910 Q=1 T=11.55 QID:124 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S32 Ask P=095 Q=1 T= 2.66 QID:None OID:-3.14]
Trader Quote: [S32 Ask P=910 Q=1 T=11.55 QID:124 OID:-3.14]

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [B36 Bid P=105 Q=1 T= 6.33 QID:None OID:-3.14]
Trader Quote: [B36 Bid P=105 Q=1 T=11.56 QID:125 OID:-3.14]

trial0001;  
replay True SHVR   0.0 []
Trader Quote: [B36 Bid P=105 Q=1 T= 6.33 QID:None OID:-3.14]
Trader Quote: [B36 Bid P=105 Q=1 T=11.56 QID:125 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False SHVR   0.0 []
Trader Quote: [S26 Ask P=096 Q=1 T= 9.77 QID:None OID:-3.14]
Trader Quote: [S26 Ask 

...

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S32 Ask P=095 Q=1 T= 2.66 QID:None OID:-3.14]
Trader Quote: [S32 Ask P=133 Q=1 T=39.99 QID:635 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S32 Ask P=095 Q=1 T= 2.66 QID:None OID:-3.14]
Trader Quote: [S32 Ask P=133 Q=1 T=39.99 QID:635 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S12 Ask P=095 Q=1 T= 4.79 QID:None OID:-3.14]
Trader Quote: [S12 Ask P=390 Q=1 T=40.01 QID:636 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S12 Ask P=095 Q=1 T= 4.79 QID:None OID:-3.14]
Trader Quote: [S12 Ask P=390 Q=1 T=40.01 QID:636 OID:-3.14]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B07 Bid P=107 Q=1 T=23.92 QID:None OID:-3.14]
Trader


trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T=10.85 QID:None OID:-3.14]
Trader Quote: [S11 Ask P=634 Q=1 T=42.56 QID:704 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T=10.85 QID:None OID:-3.14]
Trader Quote: [S11 Ask P=634 Q=1 T=42.56 QID:704 OID:-3.14]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T=10.85 QID:None OID:-3.14]
Trader Quote: [S11 Ask P=785 Q=1 T=42.58 QID:705 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T=10.85 QID:None OID:-3.14]
Trader Quote: [S11 Ask P=785 Q=1 T=42.58 QID:705 OID:-3.14]

trial0001;  
replay False SHVR   0.0 [{'time': 12.162499999999843, 'price': 105, 'type': 'Trade', 'party1': 'B06', 'qty': 1, 'party2': 'S25'}]
Trader Quote: [B06 Bid P=108 Q=1 T=40.76 QID:None OID:-3.14]
Trader Quote: [B06 Bid P=108 Q=1 T=42.59 QID:706 OID:-3.14]

trial0001;  
replay True SHVR   0.0 [{'time': 12.162499999999843, 'p2_qid': 136.0, 'price': 


trial0001;  

trial0001;  

trial0001;  
replay False GVWY   9.0 [{'time': 26.66249999999902, 'price': 98, 'type': 'Trade', 'party1': 'S07', 'qty': 1, 'party2': 'B31'}]
Trader Quote: [B31 Bid P=107 Q=1 T=31.49 QID:None OID:-3.14]
Trader Quote: [B31 Bid P=107 Q=1 T=45.09 QID:764 OID:-3.14]

trial0001;  
replay True GVWY   9.0 [{'time': 26.66249999999902, 'p2_qid': 364.0, 'price': 98, 'type': 'Trade', 'party1': 'S07', 'qty': 1, 'p1_qid': 328, 'party2': 'B31'}]
Trader Quote: [B31 Bid P=107 Q=1 T=31.49 QID:None OID:-3.14]
Trader Quote: [B31 Bid P=107 Q=1 T=45.09 QID:764 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [B21 Bid P=108 Q=1 T=43.51 QID:None OID:-3.14]
Trader Quote: [B21 Bid P=076 Q=1 T=45.15 QID:765 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B21 Bid P=108 Q=1 T=43.51 QID:None OID:-3.14]
Trader Quote: [B21 Bid P=076 Q=1 T=45.15 Q

Trader Quote: [B27 Bid P=108 Q=1 T=36.86 QID:None OID:-3.14]
Trader Quote: [B27 Bid P=084 Q=1 T=47.13 QID:823 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [B27 Bid P=108 Q=1 T=36.86 QID:None OID:-3.14]
Trader Quote: [B27 Bid P=084 Q=1 T=47.13 QID:823 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False ZIC   10.0 [{'time': 32.61249999999885, 'price': 107, 'type': 'Trade', 'party1': 'B05', 'qty': 1, 'party2': 'S22'}]
Trader Quote: [S22 Ask P=098 Q=1 T=42.12 QID:None OID:-3.14]
Trader Quote: [S22 Ask P=465 Q=1 T=47.18 QID:824 OID:-3.14]

trial0001;  
replay True ZIC   10.0 [{'time': 32.61249999999885, 'p2_qid': 490.0, 'price': 107, 'type': 'Trade', 'party1': 'B05', 'qty': 1, 'p1_qid': 487, 'party2': 'S22'}]
Trader Quote: [S22 Ask P=098 Q=1 T=42.12 QID:None OID:-3.14]
Trader Quote: [S22 Ask P=465 Q=1 T=47.18 QID:824 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay

replay False ZIC   0.0 []
Trader Quote: [S12 Ask P=095 Q=1 T= 4.79 QID:None OID:-3.14]
Trader Quote: [S12 Ask P=587 Q=1 T=49.08 QID:872 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S12 Ask P=095 Q=1 T= 4.79 QID:None OID:-3.14]
Trader Quote: [S12 Ask P=587 Q=1 T=49.08 QID:872 OID:-3.14]

trial0001;  
replay False ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T=10.85 QID:None OID:-3.14]
Trader Quote: [S11 Ask P=124 Q=1 T=49.09 QID:873 OID:-3.14]

trial0001;  
replay True ZIC   0.0 []
Trader Quote: [S11 Ask P=096 Q=1 T=10.85 QID:None OID:-3.14]
Trader Quote: [S11 Ask P=124 Q=1 T=49.09 QID:873 OID:-3.14]

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  

trial0001;  
replay False GVWY   0.0 [{'time': 9.724999999999982, 'price': 105, 'type': 'Tr